In [1]:
import os
import torch
import numpy as np
import torchvision as tv
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import (
    LearningRateMonitor,
    ModelCheckpoint,
    EarlyStopping,
)
# from torch.utils.data import DataLoader, random_split, Dataset, WeightedRandomSampler
# from torchvision.datasets import ImageFolder
# from PIL import Image

from src.utils.helpers import load_config
from src.training.dataset import ImageDataModule
from src.models.classification_model import ImageClassifier

In [2]:
real_filepaths = os.listdir("src/data/chameleon/real")
fake_filepaths = os.listdir("src/data/chameleon/fake")

real_filepaths = [os.path.join("src/data/chameleon/real", fp) for fp in real_filepaths]
fake_filepaths = [os.path.join("src/data/chameleon/fake", fp) for fp in fake_filepaths]

random_real_filepaths = np.random.choice(real_filepaths, size=2000, replace=False)
random_fake_filepaths = np.random.choice(fake_filepaths, size=2000, replace=False)

real_count = len(real_filepaths)
fake_count = len(fake_filepaths)
print(f"REAL COUNT: {real_count} ({round(real_count/(fake_count+real_count) * 100, 1)}%)")
print(f"FAKE COUNT: {fake_count} ({round(fake_count/(fake_count+real_count) * 100, 1)}%)")

REAL COUNT: 14863 (57.1%)
FAKE COUNT: 11170 (42.9%)


In [3]:
torch.set_float32_matmul_precision("high")
config = load_config("src/configs/chameleon_convnext_tiny.yaml")
pl.seed_everything(config.get("seed"))

csv_logger = CSVLogger(
    save_dir=config["logging"].get("logs_dir", "logs"),
    name=config.get("experiment_name"),
)
callbacks = []
data_module = ImageDataModule(config)

Seed set to 7643


In [4]:
# Enable learning rate monitoring hook:
if config["training"].get("lr_monitoring"):
    monitor_lr = LearningRateMonitor(logging_interval="epoch")
    callbacks.append(monitor_lr)

# Enable early stopping hook:
estop = config["training"].get("early_stopping", {})
if estop.get("enabled"):
    early_stopping = EarlyStopping(
        monitor=estop.get("monitor", "val_loss"),
        patience=estop.get("patience", 5),
        mode=estop.get("mode", "min"),
        verbose=True,
    )
    callbacks.append(early_stopping)

# Enable model checkpointing hook:
checkpoint = ModelCheckpoint(
    dirpath=config["logging"].get("checkpoint_dir", "checkpoints"),
    filename="{epoch:02d}-{val_acc:.2f}",
    save_top_k=1,
    monitor="val_acc",
    mode="max",
    save_last=True,
)
callbacks.append(checkpoint)

In [5]:
# Initialize model:
net = ImageClassifier(config)

# Configure training session:
trainer = pl.Trainer(
    min_epochs=config["training"].get("min_epochs"),
    max_epochs=config["training"].get("max_epochs", -1),
    logger=csv_logger,
    enable_checkpointing=True,
    enable_progress_bar=True,
    enable_model_summary=True,
    devices="auto",
    accelerator="auto",
    log_every_n_steps=5,
    callbacks=callbacks,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
# Begin training:
trainer.fit(
    model=net,
    datamodule=data_module,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | loss_func | CrossEntropyLoss | 0      | train
1 | backbone  | ConvNeXt         | 27.8 M | train
-------------------------------------------------------
27.8 M    Trainable params
0         Non-trainable params
27.8 M    Total params
111.287   Total estimated model params size (MB)
250       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.110


...

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved by 0.044 >= min_delta = 0.0. New best score: 0.048


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 2 records. Best score: 0.048. Signaling Trainer to stop.
`Trainer.fit` stopped: `max_epochs=5` reached.


In [7]:
# Test best model:
trainer.test(
    model=net,
    datamodule=data_module,
    ckpt_path="src/checkpoints/last.ckpt",
)

Restoring states from the checkpoint path at src/checkpoints/last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at src/checkpoints/last.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9930849075317383     │
│         test_loss         │   0.028118370100855827    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.028118370100855827, 'test_acc': 0.9930849075317383}]